# 水库模型 (Reservoir Model)

`水库 (Reservoir)` 类模拟一个简单的水库。其主要功能是根据入流和出流来追踪蓄水量。水位由水量和固定的表面积计算得出。本Jupyter Notebook演示了如何设置和运行一个带有 `Reservoir` 模型的仿真。

## 状态变量

水库的状态由以下变量定义：

- `volume` (float): 水库当前的蓄水量 (m³)。
- `water_level` (float): 水位，计算方式为水量除以表面积 (m)。
- `outflow` (float): 水库的出流量 (m³/s)，由下游需求决定。

## 参数

水库的物理属性通过以下参数设置：

- `surface_area` (float): 水库的表面积 (m²)。

## 仿真示例

以下代码模拟了一个水库在几天内的变化。它从一个初始水量开始，并接收恒定的入流。出流量在仿真过程中会变化，以模拟变化的需求。仿真跟踪了水库的水量和水位，然后将它们绘制成图表。

**注意:** 要运行此Notebook，请确保您已安装 `matplotlib` (`pip install matplotlib`)，并且您是从项目的根目录运行Jupyter服务器的。

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.core.interfaces import State, Parameters

# 水库的初始状态和参数
initial_state = State(volume=2e7, water_level=0.0, outflow=0.0)
parameters = Parameters(surface_area=2e6) # 2 平方公里

# 创建一个Reservoir实例
reservoir = Reservoir(name="main_reservoir", initial_state=initial_state, parameters=parameters)

# 仿真设置
dt = 3600  # 时间步长（秒）(1小时)
simulation_duration = 3600 * 24 * 10  # 10天
num_steps = int(simulation_duration / dt)
inflow_rate = 20.0  # 恒定入流 (m³/s)

# 存储结果用于绘图
history = []

# 运行仿真循环
for t in range(num_steps):
    # 模拟变化的出流需求
    if (t // 24) % 2 == 0: # 每隔一天交替需求
        requested_outflow = 25.0
    else:
        requested_outflow = 15.0
        
    action = {'outflow': requested_outflow}
    
    reservoir.set_inflow(inflow_rate)
    current_state = reservoir.step(action=action, dt=dt)
    history.append(current_state.copy())

# 打印最终状态
print("最终状态:", reservoir.get_state())

# 准备绘图数据
time_in_days = [i * dt / (3600*24) for i in range(num_steps)]
volumes = [s['volume'] / 1e6 for s in history] # 单位：百万立方米
water_levels = [s['water_level'] for s in history]

# 绘制结果
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(time_in_days, volumes, label='水量 (Volume)')
ax1.set_ylabel('水量 (百万 m³)')
ax1.set_title('水库仿真')
ax1.grid(True)
ax1.legend()

ax2.plot(time_in_days, water_levels, label='水位 (Water Level)', color='orange')
ax2.set_xlabel('时间 (天)')
ax2.set_ylabel('水位 (m)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
